<a href="https://colab.research.google.com/github/Cherevatin/Algorithm-theory/blob/main/cifar_10_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy


from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.metrics import accuracy_score


# Устанавливаем seed для повторяемости результатов
numpy.random.seed(42)

# Размер изображения
img_rows, img_cols = 32, 32

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Преобразование размерности изображений
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)


# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)


MODELS_COUNT = 5;
y_pred = None

for i in range (MODELS_COUNT):
  # Создаем последовательную модель
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(32,32,3),padding = "same"))
  model.add(Conv2D(64, kernel_size=(4, 4),
                  activation='relu',
                   padding = "same"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu',padding = "same"))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(Dropout(0.2)) 
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))


  # Компилируем модель
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  print(model.summary())
  
  # Обучаем сеть
  model.fit(X_train, Y_train, batch_size=200, epochs=25, validation_split=0.1, verbose=2)

  # Оцениваем качество обучения сети на тестовых данных
  scores = model.evaluate(X_test, Y_test, verbose=0)
  print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

  if y_pred is None:
    y_pred = model.predict(X_test)
  else:
    y_pred += model.predict(X_test)

y_pred /= MODELS_COUNT
pred = numpy.argmax(y_pred, axis=1)
acc_test = accuracy_score(y_test, pred)

print("Final accuracy: {}%".format(round(acc_test*100,2)))


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 32, 32, 64)        32832     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
dropout_31 (Dropout)         (None, 14, 14, 128)     